In [1]:
import uuid
from functools import partial

import pandas as pd
import geopandas as gpd
import mesa
import mesa_geo as mg
from shapely.geometry import Point

In [2]:
import os
print(os.getcwd()) 

/home/emoreno/Projects/agents-and-networks-in-python/notebooks


In [3]:
campus = "ub"
data_file_prefix = "UB"
campus_params = {
        "ub": {"data_crs": "epsg:4326", "commuter_speed": 0.5},
        "gmu": {"data_crs": "epsg:2283", "commuter_speed": 0.4},
    }
buildings_file = f"../data/raw/{campus}/{data_file_prefix}_bld.shp"
walkway_file = f"../data/raw/{campus}/{data_file_prefix}_walkway_line.shp"
data_crs = campus_params[campus]["data_crs"]

assert campus in ("ub", "gmu")

buildings_df = gpd.read_file("../data/raw/ub/UB_bld.shp")

if campus == "gmu":
    buildings_df.fillna(0.0, inplace=True)
    buildings_df.rename(columns={"NAME": "name"}, inplace=True)
buildings_df.drop("Id", axis=1, inplace=True)
buildings_df.index.name = "unique_id"
buildings_df = buildings_df.set_crs(data_crs, allow_override=True).to_crs(
    data_crs
)
buildings_df["centroid"] = [
    (x, y) for x, y in zip(buildings_df.centroid.x, buildings_df.centroid.y)
]

walkway_df = (
            gpd.read_file("../data/raw/ub/UB_walkway_line.shp")
            .set_crs(data_crs, allow_override=True)
            .to_crs(data_crs)
        )

walkway_df.head()

/tmp/ipykernel_1893835/1018979763.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  (x, y) for x, y in zip(buildings_df.centroid.x, buildings_df.centroid.y)


,Id,geometry
0,0,"LINESTRING (-78.78608 43.00990, -78.78623 43.0..."
1,0,"LINESTRING (-78.78682 43.00841, -78.78675 43.0..."
2,0,"LINESTRING (-78.78749 43.00956, -78.78733 43.0..."
3,0,"LINESTRING (-78.78732 43.00908, -78.78736 43.0..."
4,0,"LINESTRING (-78.78316 43.00903, -78.78307 43.0..."


In [4]:
import matplotlib.pyplot as plt
from pyrosm import OSM
from pyrosm import get_data

# We open our trimmed Santiago Metropolitan Region PBF file and check if pyrosm loaded it successfully.
osm = OSM("../data/external/OSM/chile-rm-santiago-latest.osm.pbf")
print("Type of 'osm' instance: ", type(osm))
buildings_df = osm.get_buildings()
buildings_df.set_index("id", inplace=True)
buildings_df.index.name = "unique_id"

Type of 'osm' instance:  <class 'pyrosm.pyrosm.OSM'>


In [5]:

buildings_df["centroid"] = [
    (x, y) for x, y in zip(buildings_df.centroid.x, buildings_df.centroid.y)
]

buildings_df = buildings_df[["geometry", "centroid"]]

buildings_df.head()

/tmp/ipykernel_1893835/3164136943.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  (x, y) for x, y in zip(buildings_df.centroid.x, buildings_df.centroid.y)


,geometry,centroid
unique_id,,
24964120,"POLYGON ((-70.65152 -33.43776, -70.65168 -33.4...","(-70.65187357573636, -33.43764282947849)"
24964133,"POLYGON ((-70.65122 -33.43728, -70.65087 -33.4...","(-70.6510858979376, -33.4370227067583)"
24964134,"POLYGON ((-70.65056 -33.43685, -70.65055 -33.4...","(-70.6502555566452, -33.43698018829323)"
24964144,"POLYGON ((-70.65116 -33.44359, -70.65158 -33.4...","(-70.65143138870809, -33.44337005361236)"
24964188,"POLYGON ((-70.65154 -33.43417, -70.65132 -33.4...","(-70.65105672607619, -33.433794182575646)"


In [6]:
walkway_df = osm.get_network("walking")

In [7]:
walkway_df = walkway_df.explode(ignore_index = True).reset_index()[["geometry"]]
walkway_df.index.name = "unique_id"
walkway_df.head()

,geometry
unique_id,
0,"LINESTRING (-70.65020 -33.43488, -70.65024 -33..."
1,"LINESTRING (-70.63704 -33.43686, -70.63685 -33..."
2,"LINESTRING (-70.63685 -33.43689, -70.63679 -33..."
3,"LINESTRING (-70.63679 -33.43690, -70.63673 -33..."
4,"LINESTRING (-70.63673 -33.43695, -70.63670 -33..."


In [8]:
import networkx as nx

nodes, edges = osm.get_network(nodes=True, network_type="walking")
G = osm.to_graph(nodes, edges, graph_type="networkx")

#nx_graph = G.subgraph(max(nx.connected_components(G), key=len))

In [9]:
nodes.head()

,lon,lat,tags,timestamp,version,changeset,id,geometry
0,-70.650197,-33.434880,"{'highway': 'traffic_signals', 'traffic_signal...",1678225286,1,0,10719909687,POINT (-70.65020 -33.43488)
1,-70.650236,-33.434635,{'highway': 'traffic_signals'},1573275019,4,0,6214791292,POINT (-70.65024 -33.43464)
2,-70.637041,-33.436863,None,1574562242,3,0,5375628055,POINT (-70.63704 -33.43686)
3,-70.636850,-33.436889,None,1608355382,2,0,7005397328,POINT (-70.63685 -33.43689)
4,-70.636785,-33.436904,None,1608355382,3,0,5375628054,POINT (-70.63679 -33.43690)


In [11]:
a = []
for n in G.nodes.data():
    a.append((n[0], n[1]["x"], n[1]["y"]))

a

[(10719909687, -70.6501973, -33.4348796),
 (6214791292, -70.6502356, -33.4346355),
 (5375628055, -70.6370411, -33.4368632),
 (7005397328, -70.63685, -33.4368894),
 (5375628054, -70.636785, -33.4369036),
 (5375628053, -70.6367274, -33.4369511),
 (130334876, -70.6367036, -33.437011),
 (8242018345, -70.6366809, -33.4371111),
 (7005397327, -70.6366241, -33.4373607),
 (10173365609, -70.6366038, -33.4374275),
 (2816330929, -70.6365782, -33.437512),
 (10719909689, -70.650271, -33.4348894),
 (10719909690, -70.6503418, -33.4348988),
 (7570208832, -70.6515279, -33.4350313),
 (386310, -70.6516095, -33.4350397),
 (7570208834, -70.6516956, -33.4350506),
 (386301, -70.6529483, -33.4352084),
 (605183940, -70.6534131, -33.4352738),
 (7608250562, -70.6543151, -33.4354007),
 (4444430690, -70.6555177, -33.432377),
 (9301900027, -70.6555839, -33.4323797),
 (9301900026, -70.6556496, -33.4323874),
 (6284820632, -70.6557141, -33.4323999),
 (9301900025, -70.6557807, -33.4324185),
 (2195161754, -70.6558448, -3

In [93]:
import numpy as np

xlist = list(G.nodes.data("x"))
ylist = list(G.nodes.data("y"))

#thelist = np.append(xlist, ylist, axis=1)
#thelist = np.delete(thelist, [2], axis=1)

xlist

[(10719909687, -70.6501973),
 (6214791292, -70.6502356),
 (5375628055, -70.6370411),
 (7005397328, -70.63685),
 (5375628054, -70.636785),
 (5375628053, -70.6367274),
 (130334876, -70.6367036),
 (8242018345, -70.6366809),
 (7005397327, -70.6366241),
 (10173365609, -70.6366038),
 (2816330929, -70.6365782),
 (10719909689, -70.650271),
 (10719909690, -70.6503418),
 (7570208832, -70.6515279),
 (386310, -70.6516095),
 (7570208834, -70.6516956),
 (386301, -70.6529483),
 (605183940, -70.6534131),
 (7608250562, -70.6543151),
 (4444430690, -70.6555177),
 (9301900027, -70.6555839),
 (9301900026, -70.6556496),
 (6284820632, -70.6557141),
 (9301900025, -70.6557807),
 (2195161754, -70.6558448),
 (5587003805, -70.6487208),
 (462794274, -70.648736),
 (3306878389, -70.648778),
 (6214791300, -70.6449102),
 (5933052999, -70.6449525),
 (6010654876, -70.6449846),
 (5933052998, -70.6450014),
 (10722370475, -70.6450107),
 (6214791302, -70.6450988),
 (10685413063, -70.6451145),
 (102381378, -70.6451221),
 (59

In [13]:
from sklearn.neighbors import KDTree

kd_tree = KDTree(nodes[["lon", "lat"]])